# Cuaderno 20: Caminos más cortos con duración acotada

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido $D=(V,A)$; 
* un vector de costos $c \in \ZZ^{A}$  asociados a los arcos de $D$; 
* un vector de tiempos de tránsito $t \in \ZZ^{A}_{+}$  asociados a los arcos de $D$; 
* un nodo de salida $r$;
* un nodo de llegada $s$, y
* una cota $B_{\max} \in \ZZ$ para la duración máxima permitida.

El *problema del camino más corto con duración acotada* consiste en encontrar un camino $P$ desde $r$ hasta $s$ cuyo costo sea mínimo y cuya duración no supere la cota $B_{max}$ establecida. El costo de un camino se calcula sumando de los costos de sus arcos, mientras que la duración es la suma de los tiempos de tránsito de los mismos.

Utilizaremos la siguientes variables de decisión: 
* variables binarias $x_{ij}$, para indicar si los arcos de $A$ son seleccionados o no dentro de $P$; y, 
* variables no negativas $B_i$ para indicar la duración del subcamino de $P$ que va desde $r$ hasta $i$, en caso de que el nodo $i$ sea visitado por $P$ (si $i$ no es visitado por $P$, el valor de $B_i$ es irrelevante para el modelo).

De esta manera, se puede formular al problema como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(r, i) \in A} x_{ri} = 1,\\
&\sum_{(i, s) \in A} x_{is} = 1,\\
& \sum_{(j, i) \in A} x_{ji} - \sum_{(i, j) \in A} x_{ij} = 0, \quad \forall i \in V \setminus \tabulatedset{r,s},\\
&B_j \geq B_i + t_{ij} - M(1 -  x_{ij}), \quad \forall (i,j) \in A, \\ 
&B_s \leq B_{\max},\\ 
&B_i \geq 0, \quad \forall i \in V,\\
& x_{ij} \in \tabulatedset{0, 1}, \quad \forall (i, j) \in A.
\end{align*}

$M$ es una constante con un valor suficientemente grande como para que la restricción $B_j \geq B_i + t_{ij} - M$ sea siempre redundante. Por ejemplo, en nuestro problema puede tomarse $M:=\sum_{ij \in A} t_{ij}$. 
Observar que este modelo funciona aún si el grafo tiene circuitos de costo negativo, siempre y cuando los tiempos de tránsito sean positivos.

Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los conjuntos y parámetros del modelo:

In [1]:
from gurobipy import *

# Arcos, costos y tiempos de tránsito 
A, c, t = multidict({
  (1, 2):  (3, 2), 
  (1, 3):  (20,1),
  (2, 3):  (10,2),
  (2, 4):  (5,2),
  (4, 3):  (2,3),
  (3, 5):  (2,3),
  (4, 5):  (5,1),
  (4, 6):  (10,1),
  (5, 6):  (2,3),
  (5, 7):  (-3,1),
  (7, 8):  (-3,1),  
  (8, 5):  (-3,1)})

# Recuperar los nodos del grafo a partir de los extremos de los arcos
V = tuplelist(set([i for (i,j) in A] + [j for (i,j) in A]))

# Nodo de salida
r = 1

# Nodo de llegada
s = 6

# Duración máxima
Bmax = 10

# Constante suficientemente grande:
M = quicksum([t[i,j] for (i,j) in A])

# --- los valores a partir de aqui se calculan automaticamente ---
# nodos internos: Vi := V \ {s, t}
Vi = tuplelist([i for i in V if i!=r and i!=s])

Creamos ahora el objeto modelo y las variables de binarias de selección de arcos $x_{ij}$. Observar que las variables están indexadas por los conjuntos de arcos.

In [14]:
# Crear el objeto modelo
m = Model('cheapest-path-bounded-duration')

# Crear las variables de selección de arcos
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# Crear las variables de tiempo de visita en los nodos
B = m.addVars(V, name="B")


Definimos la función objetivo a minimizar:

In [15]:
m.setObjective(x.prod(c, '*'), GRB.MINIMIZE)

Definimos las restricciones del modelo:

1. Restricciones de grado y de conservación de flujo

In [16]:
# Arcos salientes de s
m.addConstr(x.sum(r,'*')  == 1, "grado_r")

# Arcos entrantes a t
m.addConstr(x.sum('*', s)  == 1, "grado_s")

# Balance de grados en los demas nodos
m.addConstrs(
    (x.sum('*',i) - x.sum(i,'*')  == 0 for i in Vi), "grado")

{2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>}

2. Restricciones de tiempo de visita en los nodos:

In [17]:
# Bj >= Bi + tij si el arco (i,j) forma parte del camino:
m.addConstrs((B[j] >= B[i] + t[i,j] - M*(1 - x[i,j]) for (i,j) in A), "tiempos_visita")

# Duración máxima del camino:
m.addConstr(B[s]  <= Bmax, "dur_max")

# Alternativa 2 (no funciona si el grafo tiene circuitos de costo negativo)
# m.addConstr(x.prod(t, '*')  <= Bmax, "dur_max")

<gurobi.Constr *Awaiting Model Update*>

Resolvemos el modelo y mostramos la solución. Notar que los valores de las variables pueden recuperarse también empleando el método `getAttr`:

In [18]:
# Calcular la solucion optima
m.optimize()

# Escribir la solucion
if m.status == GRB.Status.OPTIMAL:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    # vB = m.getAttr('x', B)
    print('Arcos seleccionados:')
    Bs = 0
    for i,j in A:
        if vx[i,j] >= 0.99:
            print('{} -> {}'.format(i, j))
            Bs += t[i,j]
    # Mostrar duración:
    print("Duración: {}".format(Bs))
    print("Costo: {}".format(m.objval))

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 21 rows, 20 columns and 61 nonzeros
Model fingerprint: 0xf7a8ae93
Variable types: 8 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 18.0000000
Presolve removed 21 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.11 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 15 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.500000000000e+01, best bound 1.500000000000e+01, gap 0.0000%
Arcos seleccionados:
1 -> 2
2 -> 4
4 -> 5
5 -> 6
Duración: 8
Costo: 15.0


Finalmente, grafiquemos la solución empleando `networkx` y `ipycytoscape`:

In [19]:
import networkx as nx
import ipycytoscape
D = nx.DiGraph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['etiq']= str(i)
D.add_edges_from(A)
for i,j in A:
    D.edges[i,j]['etiq'] = '({}, {})'.format(c[i,j], t[i,j])
    D.edges[i,j]['color'] =  '#9dbaea' if vx[i,j]<=0.1 else '#ff007f'
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=True)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 4, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'background…

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [27]:
# Implementacion de modelos de programacion lineal entera
# Problema de caminos mas cortos con duración acotada
# (cheapest path with bounded duration)

# Luis M. Torres (EPN 2020)

from gurobipy import *

# Arcos, costos y tiempos de tránsito 
A, c, t = multidict({
  (1, 2):  (3, 2), 
  (1, 3):  (20,1),
  (2, 3):  (10,2),
  (2, 4):  (5,2),
  (4, 3):  (2,3),
  (3, 5):  (2,3),
  (4, 5):  (5,1),
  (4, 6):  (10,1),
  (5, 6):  (2,3),
  (5, 7):  (-3,1),
  (7, 8):  (-3,1),  
  (8, 5):  (-3,1)})

# Recuperar los nodos del grafo a partir de los extremos de los arcos
V = tuplelist(set([i for (i,j) in A] + [j for (i,j) in A]))

# Nodo de salida
r = 1

# Nodo de llegada
s = 6

# Duración máxima
Bmax = 10

# Constante suficientemente grande:
M = quicksum([t[i,j] for (i,j) in A])

# --- los valores a partir de aqui se calculan automaticamente ---
# nodos internos: Vi := V \ {s, t}
Vi = tuplelist([i for i in V if i!=r and i!=s])

try:
    # Crear el objeto modelo
    m = Model('cheapest-path-bounded-duration')

    # Crear las variables de selección de arcos
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # Crear las variables de tiempo de visita en los nodos
    B = m.addVars(V, name="B")

    # Definir la funcion objetivo
    m.setObjective(x.prod(c, '*'), GRB.MINIMIZE)

    # Arcos salientes de r
    m.addConstr(x.sum(r,'*')  == 1, "grado_r")

    # Arcos entrantes a s
    m.addConstr(x.sum('*', s)  == 1, "grado_s")

    # Balance de grados en los demas nodos
    m.addConstrs(
        (x.sum('*',i) - x.sum(i,'*')  == 0 for i in Vi), "grado")

    # Bj >= Bi + tij si el arco (i,j) forma parte del camino:
    m.addConstrs((B[j] >= B[i] + t[i,j] - M*(1 - x[i,j]) for (i,j) in A), "tiempos_visita")

    # Duración máxima del camino:
    m.addConstr(B[s]  <= Bmax, "dur_max")

    # Calcular la solución óptima
    m.optimize()

    # Escribir la solucion
    if m.status == GRB.Status.OPTIMAL:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        vB = m.getAttr('x', B)
        print('Arcos seleccionados:')
        Bs = 0
        for i,j in A:
            if vx[i,j] >= 0.1:
                print('{} -> {}'.format(i, j))
                Bs += t[i,j]
        # Mostrar duración:
        print("Duración: {}".format(Bs))
        print("Costo: {}".format(m.objval))            
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 21 rows, 20 columns and 61 nonzeros
Model fingerprint: 0xf7a8ae93
Variable types: 8 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 18.0000000
Presolve removed 21 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 15 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.500000000000e+01, best bound 1.500000000000e+01, gap 0.0000%
Arcos seleccionados:
1 -> 2
2 -> 4
4 -> 5
5 -> 6
Duración: 8
Costo: 15.0
